In [42]:
from sympy import Ellipse, Point, Rational
import itertools
from scipy.optimize import minimize

In [9]:
from sympy.solvers import solve
from sympy import Symbol

In [1]:
def ellipse_base(b1, b2, rssi1, rssi2):
    # ellipse de la forme X^2 + Y^2 + x_coeff*X + y_coeff*Y +  const_coeff = 0
    # bi sont les coordonnées des bases i
    x_coeff = (-2*rssi2/(rssi2-rssi1))*(b1[0]-b2[0]*rssi2/rssi1)
    y_coeff = (-2*rssi2/(rssi2-rssi1))*(b1[1]-b2[1]*rssi2/rssi1)
    const_coeff = ((b1[0]+b1[1])*rssi2 + (b2[0]+b2[1])*rssi1)/(rssi2-rssi1)
    return [x_coeff, y_coeff, const_coeff]

In [3]:
def distance(ellipse, X):
    # calcul de la plus petite distance entre le point X et l'ellipse d'equation type ellipse_base
    # on utilisera le solveur pour obtenir le point de l'ellipse apres avoir dérivé le schoulblique
    x1 = Symbol('x_1')
    y1 = Symbol('y_1')
    x2 = Symbol('x_2')
    y2 = Symbol('y_2')
    x1 = X[0]
    y1 = X[1]
    sol solve((diff((x2-x1)**2+(y2-y1)**2, x2) = 0, 
          x2**2 + y2**2 + ellipse[0]*x2 + ellipse[1]*y2 + ellipse[3] = 0), 
          x2, y2)
    distance = (sol[x2]-x1)**2+(sol[y2]-y1)**2
    return distance

In [ ]:
def list_3_ellipses(b, rssi):
    # on entre en entree un vecteur 
    # on fait une liste contenant les 3 ellipse
    liste_ellipses = []
    for i, j in itertools.combinations(range(0,3)):
        liste_ellipses += [ellipse_base(b[i], b[j], rssi[i], rssi[j])]
    return liste_ellipses

In [43]:
def sum_disdances(X, liste_ellipses):
    result = 0
    for i in range(0,3):
        result += distance(liste_ellipses[i], X)
    return result

In [44]:
def get_coordinates(b1, b2, b3, rssi1, rssi2, rssi3):
    b = [b1, b2, b3]
    rssi = [rssi1, rssi2, rssi3]
    liste_ellipses = list_3_ellipses(b, rssi)
    X = optimize(sum_disdances, [1, 1], method='Nelder-Mead', 
                 args = (liste_ellipses), 
                 options={'xtol': 1e-5}).x
    return X